In [23]:
import curl_cffi
from curl_cffi import requests
import scraper_functions
from scraper_functions import VPNTester
import random, time
import selectolax
from selectolax.parser import HTMLParser
import pandas as pd
import numpy as np
from datetime import datetime
from zoneinfo import ZoneInfo

##### VPN Checker

In [24]:
tester = VPNTester()
tester.vpn_check()

[🛡️  VPN Check] Your current IP: 185.239.150.22


##### Fixing impersonate agent for the session

In [25]:
FIXED_IMPERSONATE = random.choice(scraper_functions.IMPERSONATE_OPTIONS)
print(FIXED_IMPERSONATE)

chrome99


# Categorias

In [26]:
url = 'https://www.superseis.com.py/default.aspx'

response = curl_cffi.requests.get(url, impersonate = FIXED_IMPERSONATE)

In [27]:
html = response.text
tree = HTMLParser(html)

In [28]:
data = []

# Buscar todos los nodos de nivel 1
for lvl1_li in tree.css("li.level1"):
    lvl1_a = lvl1_li.css_first("a")
    if not lvl1_a:
        continue
    lvl1_name = lvl1_a.text(strip=True)

    # Dentro de este <li>, buscar los hijos de nivel 2
    for lvl2_li in lvl1_li.css("ul > li.level2"):
        lvl2_a = lvl2_li.css_first("a")
        if not lvl2_a:
            continue
        lvl2_name = lvl2_a.text(strip=True)

        # Dentro del lvl2, buscar los hijos de nivel 3 (con enlaces)
        for lvl3_li in lvl2_li.css("ul > li.level3"):
            lvl3_a = lvl3_li.css_first("a[href]")
            if not lvl3_a:
                continue
            lvl3_name = lvl3_a.text(strip=True)
            lvl3_url = lvl3_a.attributes.get("href")

            data.append({
                "categoria_nivel_1": lvl1_name,
                "categoria_nivel_2": lvl2_name,
                "categoria_nivel_3": lvl3_name,
                "url": lvl3_url,
            })

In [29]:
df_categorias = pd.DataFrame(data)
df_categorias.head()
df_categorias.nunique()

categoria_nivel_1     19
categoria_nivel_2     80
categoria_nivel_3    304
url                  321
dtype: int64

# Productos (test)

In [30]:
url = 'https://www.superseis.com.py/category/3-almacen-aderezoscondimentos-aceites.aspx'

response = curl_cffi.requests.get(url, impersonate = FIXED_IMPERSONATE)

In [31]:
productos = tree.css("div.producto")

for prod in productos:
    try:
        titulo = prod.css_first("h2.product-title").text(strip=True)
        marca = prod.css_first("div.product-brand").text(strip=True)
        precio = prod.css_first("span.price-label").text(strip=True)

        print(f"{titulo} | {marca} | {precio}")
    except AttributeError:
        print("[⚠️] Algún dato no encontrado en un producto")


# Productos

In [32]:
BASE_WAIT = (2.0, 5.0)
def esperar():
    time.sleep(random.uniform(*BASE_WAIT))

In [33]:
def obtener_max_page(tree):
    paginador = tree.css("div.product-pager-box div")
    if not paginador:
        return 1
    nums = []
    for a in paginador[0].css("a"):
        try:
            nums.append(int(a.text(strip=True)))
        except ValueError:
            continue
    for span in paginador[0].css("span"):
        try:
            nums.append(int(span.text(strip=True)))
        except ValueError:
            continue
    return max(nums) if nums else 1

In [34]:
def extraer_productos(tree):
    productos = []
    for prod in tree.css("div.producto"):
        try:
            titulo = prod.css_first("h2.product-title").text(strip=True)
            marca = prod.css_first("div.product-brand").text(strip=True)
            precio = prod.css_first("span.price-label").text(strip=True)
            productos.append({
                "titulo": titulo,
                "marca": marca,
                "precio": precio
            })
        except AttributeError:
            continue
    return productos

In [35]:
# Shuffleled
urls = df_categorias["url"].unique() #[18:-2] subset for testing
np.random.shuffle(urls)
urls = urls[0:3]
urls

array(['https://www.superseis.com.py/category/163-bazar-cocina-vidrio.aspx',
       'https://www.superseis.com.py/category/760-perfumeria-cuidado-personal-hilossedas-dentales.aspx',
       'https://www.superseis.com.py/category/460-limpieza-hogar-cepillos-limpiadores.aspx'],
      dtype=object)

In [36]:
productos_final = []

# 🔁 Iterar las 5 primeras URLs
for base_url in urls:
    print(f"\n🔎 Scrapeando categoría: {base_url}")
    esperar()

    response = requests.get(base_url, impersonate=FIXED_IMPERSONATE)
    tree = HTMLParser(response.text)
    max_page = obtener_max_page(tree)
    print(f"📄 Total de páginas: {max_page}")

    # Iterar todas las páginas
    for page in range(1, max_page + 1):
        url_pagina = f"{base_url}?pageindex={page}"
        print(f"➡️ Página {page}: {url_pagina}")
        esperar()

        resp = requests.get(url_pagina, impersonate=FIXED_IMPERSONATE)
        html_tree = HTMLParser(resp.text)

        productos_pagina = extraer_productos(html_tree)
        productos_final.extend(productos_pagina)

print(f"\n✅ Total productos extraídos: {len(productos_final)}")


🔎 Scrapeando categoría: https://www.superseis.com.py/category/163-bazar-cocina-vidrio.aspx
📄 Total de páginas: 1
➡️ Página 1: https://www.superseis.com.py/category/163-bazar-cocina-vidrio.aspx?pageindex=1

🔎 Scrapeando categoría: https://www.superseis.com.py/category/760-perfumeria-cuidado-personal-hilossedas-dentales.aspx
📄 Total de páginas: 1
➡️ Página 1: https://www.superseis.com.py/category/760-perfumeria-cuidado-personal-hilossedas-dentales.aspx?pageindex=1

🔎 Scrapeando categoría: https://www.superseis.com.py/category/460-limpieza-hogar-cepillos-limpiadores.aspx
📄 Total de páginas: 1
➡️ Página 1: https://www.superseis.com.py/category/460-limpieza-hogar-cepillos-limpiadores.aspx?pageindex=1

✅ Total productos extraídos: 25


In [37]:
df = pd.DataFrame(productos_final)
df.head()

,titulo,marca,precio
0,VASO DE VIDRIO REF PBS53H9 LIFE Y LIVING 310 ML,,6.900
1,VASO DE VIDRIO REF GA014 LIFE Y LIVING 320 ML,,6.900
2,VASO DE VIDRIO REF 53H9 LIFE Y LIVING 310 ML,,6.900
3,VASO DE ACRILICO LARGO REF SR3002/1 SAN REMO 3...,,8.900
4,VASO RIGOLLEAU LAGRIMA 38750,,11.950


## Segunda prueba (mas completa)

In [38]:
df_categorias # = df_categorias.head(1)
#df_categorias

,categoria_nivel_1,categoria_nivel_2,categoria_nivel_3,url
0,Almacén,Aderezos/Condimentos,Aceites,https://www.superseis.com.py/category/3-almace...
1,Almacén,Aderezos/Condimentos,Aderezos/Salsas,https://www.superseis.com.py/category/4-almace...
2,Almacén,Aderezos/Condimentos,Especias,https://www.superseis.com.py/category/5-almace...
3,Almacén,Aderezos/Condimentos,Ketchup,https://www.superseis.com.py/category/6-almace...
4,Almacén,Aderezos/Condimentos,Mayonesa,https://www.superseis.com.py/category/7-almace...
...,...,...,...,...
637,Perfumeria,Protección Femenina,Toallas Higiénicas,https://www.superseis.com.py/category/776-perf...
638,Perfumeria,Rostro/Manos/Pies,Accesorios,https://www.superseis.com.py/category/734-perf...
639,Perfumeria,Rostro/Manos/Pies,Manicura/Pedicuro,https://www.superseis.com.py/category/735-perf...
640,Perfumeria,Rostro/Manos/Pies,Esmalte/Quitaesmalte,https://www.superseis.com.py/category/736-perf...


In [39]:
def extraer_productos(tree, contexto):
    productos = []
    for prod in tree.css("div.producto"):
        try:
            titulo = prod.css_first("h2.product-title").text(strip=True)
            marca = prod.css_first("div.product-brand").text(strip=True)
            precio = prod.css_first("span.price-label").text(strip=True)
            precio = prod.css_first("span.unidad-medida").text(strip=True)

            productos.append({
                "titulo": titulo,
                "marca": marca,
                "precio": precio,
                **contexto  # ← añadimos todos los metadatos de golpe
            })
        except AttributeError:
            continue
    return productos


In [40]:
INGESTION_TIME = datetime.now(ZoneInfo("America/Asuncion"))
SUPERMERCADO = "Super Seis"
productos_final = []

# 🔁 Iterar sobre cada categoría (nivel 3) con contexto
for _, row in df_categorias.iterrows():
    categoria_url = row["url"]
    cat1 = row["categoria_nivel_1"]
    cat2 = row["categoria_nivel_2"]
    cat3 = row["categoria_nivel_3"]

    print(f"\n🔎 Scrapeando categoría: {cat1} > {cat2} > {cat3}")
    esperar()

    response = requests.get(categoria_url, impersonate=FIXED_IMPERSONATE)
    tree = HTMLParser(response.text)
    max_page = obtener_max_page(tree)
    print(f"📄 Total de páginas: {max_page}")

    for page in range(1, max_page + 1):
        page_url = f"{categoria_url}?pageindex={page}"
        print(f"➡️ Página {page}: {page_url}")
        esperar()

        resp = requests.get(page_url, impersonate=FIXED_IMPERSONATE)
        html_tree = HTMLParser(resp.text)

        contexto = {
            "categoria_nivel_1": cat1,
            "categoria_nivel_2": cat2,
            "categoria_nivel_3": cat3,
            "url_categoria": categoria_url,
            "ingestion_time": INGESTION_TIME,
            "supermercado": SUPERMERCADO
        }

    productos_pagina = extraer_productos(html_tree, contexto)
    productos_final.extend(productos_pagina)



🔎 Scrapeando categoría: Almacén > Aderezos/Condimentos > Aceites
📄 Total de páginas: 2
➡️ Página 1: https://www.superseis.com.py/category/3-almacen-aderezoscondimentos-aceites.aspx?pageindex=1
➡️ Página 2: https://www.superseis.com.py/category/3-almacen-aderezoscondimentos-aceites.aspx?pageindex=2

🔎 Scrapeando categoría: Almacén > Aderezos/Condimentos > Aderezos/Salsas
📄 Total de páginas: 2
➡️ Página 1: https://www.superseis.com.py/category/4-almacen-aderezoscondimentos-aderezos.aspx?pageindex=1
➡️ Página 2: https://www.superseis.com.py/category/4-almacen-aderezoscondimentos-aderezos.aspx?pageindex=2

🔎 Scrapeando categoría: Almacén > Aderezos/Condimentos > Especias
📄 Total de páginas: 5
➡️ Página 1: https://www.superseis.com.py/category/5-almacen-aderezoscondimentos-especias.aspx?pageindex=1
➡️ Página 2: https://www.superseis.com.py/category/5-almacen-aderezoscondimentos-especias.aspx?pageindex=2
➡️ Página 3: https://www.superseis.com.py/category/5-almacen-aderezoscondimentos-especi

DNSError: Failed to perform, curl: (6) Could not resolve host: www.superseis.com.py. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

In [41]:
df = pd.DataFrame(productos_final)
df.head(10)

,titulo,marca,precio,categoria_nivel_1,categoria_nivel_2,categoria_nivel_3,url_categoria,ingestion_time,supermercado
0,ACEITE DE OLIVA EXTRA VIRGEN YBARRA 500ML,MARCA EXCLUSIVA,Un.,Almacén,Aderezos/Condimentos,Aceites,https://www.superseis.com.py/category/3-almace...,2025-07-07 08:58:42.217701-03:00,Super Seis
1,ACEITE DE OLIVA EXTRA VIRGEN YBARRA 750ML,MARCA EXCLUSIVA,Un.,Almacén,Aderezos/Condimentos,Aceites,https://www.superseis.com.py/category/3-almace...,2025-07-07 08:58:42.217701-03:00,Super Seis
2,ACEITE DE OLIVA EN BOTELLA YBARRA 500 ML,MARCA EXCLUSIVA,Un.,Almacén,Aderezos/Condimentos,Aceites,https://www.superseis.com.py/category/3-almace...,2025-07-07 08:58:42.217701-03:00,Super Seis
3,ACEITE DE OLIVA EN BOTELLA YBARRA 750 ML,MARCA EXCLUSIVA,Un.,Almacén,Aderezos/Condimentos,Aceites,https://www.superseis.com.py/category/3-almace...,2025-07-07 08:58:42.217701-03:00,Super Seis
4,ACEITE DE OLIVA EN BOTELLA YBARRA 250 ML,MARCA EXCLUSIVA,Un.,Almacén,Aderezos/Condimentos,Aceites,https://www.superseis.com.py/category/3-almace...,2025-07-07 08:58:42.217701-03:00,Super Seis
5,ACEITE PURO DE GIRASOL EN BOTELLA BIANCA 900 ML,,Un.,Almacén,Aderezos/Condimentos,Aceites,https://www.superseis.com.py/category/3-almace...,2025-07-07 08:58:42.217701-03:00,Super Seis
6,ACEITE DE OLIVA GRAB SELECCION CARBONELL 100 M...,,Un.,Almacén,Aderezos/Condimentos,Aceites,https://www.superseis.com.py/category/3-almace...,2025-07-07 08:58:42.217701-03:00,Super Seis
7,ACEITE DE OLIVA EXTRA VIRGEN EN BOTELLA OLICA ...,,Un.,Almacén,Aderezos/Condimentos,Aceites,https://www.superseis.com.py/category/3-almace...,2025-07-07 08:58:42.217701-03:00,Super Seis
8,ACEITE DE GIRASOL EN BOTELLA VICENTIN 900 ML,VICENTIN S.A.I.C,Un.,Almacén,Aderezos/Condimentos,Aceites,https://www.superseis.com.py/category/3-almace...,2025-07-07 08:58:42.217701-03:00,Super Seis
9,ACEITE DE SOJA EN BOTELLA PRIMOR 900 ML,,Un.,Almacén,Aderezos/Condimentos,Aceites,https://www.superseis.com.py/category/3-almace...,2025-07-07 08:58:42.217701-03:00,Super Seis


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4331 entries, 0 to 4330
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype                           
---  ------             --------------  -----                           
 0   titulo             4331 non-null   object                          
 1   marca              4331 non-null   object                          
 2   precio             4331 non-null   object                          
 3   categoria_nivel_1  4331 non-null   object                          
 4   categoria_nivel_2  4331 non-null   object                          
 5   categoria_nivel_3  4331 non-null   object                          
 6   url_categoria      4331 non-null   object                          
 7   ingestion_time     4331 non-null   datetime64[ns, America/Asuncion]
 8   supermercado       4331 non-null   object                          
dtypes: datetime64[ns, America/Asuncion](1), object(8)
memory usage: 304.7+ KB
